# Class Subset

In [1]:
import numpy as np
import random
import math


# turn label to subset

class Subset:

    def __init__(self, num_of_total_classes):
        self.total_classes = num_of_total_classes
        self.universe_set = set(range(0, self.total_classes))
        pai = [pow(2, i)for i in range(2, self.total_classes - 1)]
        dist_of_subset_size = pai / np.linalg.norm(pai, 1)
        self.dist_of_subset_size = dist_of_subset_size

    def index_to_subset(self, class_index):
        size_of_subset = np.random.choice(range(2, self.total_classes - 1), p=self.dist_of_subset_size)
        orig_subset = random.sample(list(self.universe_set), size_of_subset)
        if class_index in orig_subset:
            return orig_subset
        else:
            return list(self.universe_set - set(orig_subset))

    def one_hot_to_obfuscated(self, one_hot_array):
        index = one_hot_array.argmax()
        subset = self.index_to_subset(index)
        ob_array = [0] * self.total_classes
        for i in subset:
            ob_array[i] = 1/len(subset)
        return ob_array

    def index_to_obfuscated(self, index):
        subset = self.index_to_subset(index)
        ob_array = [0] * self.total_classes
        for i in subset:
            ob_array[i] = 1 / len(subset)
        return len(subset), subset, ob_array
    
    def index_to_stack_obfuscated(self, index):
        subset = self.index_to_subset(index)
        stack_y = []
        for i in subset:
          stack_y.append(self.index_to_onehot(i))
        return len(subset), subset, stack_y


    def distill(self, model_outputs, labels, max_size):
        new_label_matrix = np.zeros(shape = model_outputs.shape)
        #size_of_label = np.countnonzero(label)
        #index_of_subset = np.nonzero(label)
        for index, label in labels:
            index_of_subset = np.nonzero(label)[0]
            size_of_subset = index_of_subset.shape[0]
            if size_of_subset == max_size:
                model_output = model_outputs[index]
                a=model_output[index_of_subset].argsort()

    def index_to_onehot(self, index):
        label = [0]*self.total_classes
        label[index] = 1
        return label

    def index_to_subset_multi_hot(self, class_index):
        size_of_subset = np.random.choice(range(2, self.total_classes - 1), p=self.dist_of_subset_size)
        orig_subset = random.sample(list(self.universe_set), size_of_subset)
        if class_index in orig_subset:
            multi_hot = np.zeros(self.total_classes)
            multi_hot[orig_subset] = 1
        else:
            multi_hot = np.ones(self.total_classes)
            multi_hot[orig_subset] = 0
        return multi_hot
    
    def index_to_limited_subset(self, class_index, subset_size=2):
        union = set(range(0, self.total_classes))
        union.remove(class_index)
        subset = random.sample(list(union), subset_size-1)
        subset.append(class_index)
        #orig_subset = random.sample(list(self.universe_set), size_of_subset)
        multi_hot = np.zeros(self.total_classes)
        multi_hot[subset] = 1
        return multi_hot
        

# Scartch

In [ ]:
import numpy as np

tmp_label = [0]*5

print(tmp_label)

[0, 0, 0, 0, 0]


# New Section
Loading Cifar10, and pick 80% of pics from class0-5 as train data.

In [ ]:
import torch.nn as nn
import numpy as np
import math
from sklearn.metrics import accuracy_score
from torchvision import datasets
import torch

sub = Subset(6)

train_data = datasets.CIFAR10(root='./', train=True, download=True)
test_data = datasets.CIFAR10(root='./', train=False, download=True)

x_train = train_data.data
x_test = test_data.data
y_train = train_data.targets
y_test = test_data.targets

X_train_true = np.zeros(shape=(50000, 32, 32, 3), dtype=float)
X_test_true = np.zeros(shape=(10000, 32, 32, 3), dtype=float)

# data preparation
for index, sample in enumerate(x_train):
    X_train_true[index] = sample.astype(float) / 255
for index, sample in enumerate(x_test):
    X_test_true[index] = sample.astype(float) / 255

# split
from sklearn.model_selection import train_test_split

train_X, _, train_y, _ = train_test_split(X_train_true, y_train, test_size=0.05, random_state=42)
test_X, validation_X, test_y, validation_y = train_test_split(X_test_true, y_test, test_size=0.8, random_state=42)

test_labels = []
train_labels = []
validation_labels = []
true_labels = []
X_train = []
X_test = []
X_validation = []

X_train_0 = []
train_labels_0 = []
X_train_2 = []
train_labels_2 = []
X_train_3 = []
train_labels_3 = []
X_train_4 = []
train_labels_4 = []

for index, y in enumerate(train_y):
    if y < 6:
        size_of_subset, subset_y, obfuscated_y = sub.index_to_obfuscated(y)
        if obfuscated_y[0] > 0:
            X_train_0.append(train_X[index])
            train_labels_0.append(obfuscated_y)
        if size_of_subset == 2:
            X_train_2.append(train_X[index])
            train_labels_2.append(obfuscated_y)
        elif size_of_subset == 3:
            X_train_3.append(train_X[index])
            train_labels_3.append(obfuscated_y)
        else:
            X_train_4.append(train_X[index])
            train_labels_4.append(obfuscated_y)
        X_train.append(train_X[index])
        #train_labels.append(y)
        # size_of_subset, obfuscated_y = sub.index_to_obfuscated(y)
        train_labels.append(obfuscated_y)
        true_labels.append(y)

for index, y in enumerate(test_y):
    if y < 6:
        X_test.append(test_X[index])
        test_labels.append(y)

for index,y in enumerate(validation_y):
    if y == 0:
        X_validation.append(validation_X[index])
        validation_labels.append(y)

print(train_labels)
print(test_labels)
print(validation_labels)

batch_size = 64 


from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.utils.data as Data

train_loss = []

'''
batch_size = 16
# wrap data into torch's dataset
input_2 = torch.from_numpy(np.array(X_train_2)).to(torch.float32)
label_2 = torch.from_numpy(np.array(train_labels_2)).to(torch.float32)
torch_dataset_2 = Data.TensorDataset(input_2, label_2)
train_loader_2 = DataLoader(torch_dataset_2, batch_size=batch_size, shuffle=True)
print('2-Subset:', len(X_train_2))

batch_size = 32
# wrap data into torch's dataset
input_3 = torch.from_numpy(np.array(X_train_3)).to(torch.float32)
label_3 = torch.from_numpy(np.array(train_labels_3)).to(torch.float32)
torch_dataset_3 = Data.TensorDataset(input_3, label_3)
train_loader_3 = DataLoader(torch_dataset_3, batch_size=batch_size, shuffle=True)
print('3-Subset:', len(X_train_3))

batch_size = 64
# wrap data into torch's dataset
input_4 = torch.from_numpy(np.array(X_train_4)).to(torch.float32)
label_4 = torch.from_numpy(np.array(train_labels_4)).to(torch.float32)
torch_dataset_4 = Data.TensorDataset(input_4, label_4)
train_loader_4 = DataLoader(torch_dataset_4, batch_size=batch_size, shuffle=True)
print('4-Subset:', len(X_train_4))
'''


input = torch.from_numpy(np.array(X_train)).to(torch.float32)
label = torch.from_numpy(np.array(train_labels)).to(torch.float32)
label = torch.from_numpy(np.array(train_labels)).to(torch.float32)
torch_dataset = Data.TensorDataset(input, label)



# input = torch.from_numpy(np.array(X_train_0)).to(torch.float32)
# #label = torch.from_numpy(np.array(train_labels)).to(torch.float32)
# label = torch.from_numpy(np.array(train_labels_0)).to(torch.float32)
# torch_dataset = Data.TensorDataset(input, label)


train_loader = DataLoader(torch_dataset, batch_size=batch_size, shuffle=True)

HTTPError: ignored

# Try Ensemble way


In [2]:
import torch.nn as nn
import torch
import numpy as np
import math
from sklearn.metrics import accuracy_score
from torchvision import transforms as T
import torch.utils.data as Data
from torchvision import datasets

sub = Subset(10)

train_data = datasets.CIFAR10(root='./', train=True, download=True)
test_data = datasets.CIFAR10(root='./', train=False, download=True)

x_train = train_data.data
x_test = test_data.data
y_train = train_data.targets
y_test = test_data.targets

X_train_true = np.zeros(shape=(50000, 3, 32, 32), dtype=float)
X_test_true = np.zeros(shape=(10000, 3, 32, 32), dtype=float)

# data preparation
for index, sample in enumerate(x_train):
    X_train_true[index] = sample.reshape(3,32,32).astype(float) / 255
for index, sample in enumerate(x_test):
    X_test_true[index] = sample.reshape(3,32,32).astype(float) / 255

# split
from sklearn.model_selection import train_test_split

train_X, _, train_y, _ = train_test_split(X_train_true, y_train, test_size=0.05, random_state=42)
test_X, validation_X, test_y, validation_y = train_test_split(X_test_true, y_test, test_size=0.2, random_state=42)

print(len(validation_X))

test_labels = []
train_labels = []
X_validation = []
validation_labels = []
true_labels = []
X_train = []
X_test = []


X_train_by_class = [[], [], [], [], [], []]
train_labels_by_class = [[], [], [], [], [], []]

for index, y in enumerate(train_y):
    #size_of_subset, subset_y, obfuscated_y = sub.index_to_stack_obfuscated(y)
    multi_hot_subset = sub.index_to_limited_subset(y)
    X_train.append(train_X[index])
    train_labels.append(multi_hot_subset)
    true_labels.append(y)

for index, y in enumerate(test_y):
    X_test.append(test_X[index])
    test_labels.append(y)

for index,y in enumerate(validation_y):
    X_validation.append(validation_X[index])
    validation_labels.append(y)

print(train_labels)
print(test_labels)
print(validation_labels)



batch_size = 128


from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.utils.data as Data

train_loss = []

loader_by_class = []

normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])


transform = T.Compose([# 对原始32*32图像四周各填充4个0像素（40*40），然后随机裁剪成32*32
         T.RandomCrop(32, padding=4),
         # 按0.5的概率水平翻转图片
         T.RandomHorizontalFlip(), normalize])


input = torch.from_numpy(np.array(X_train)).to(torch.float32)
label = torch.from_numpy(np.array(train_labels)).to(torch.int64)
true_labels = torch.from_numpy(np.array(true_labels)).to(torch.long)
torch_dataset = Data.TensorDataset(input, label)
watch_dataset = Data.TensorDataset(input, true_labels)
train_loader = DataLoader(torch_dataset, batch_size=batch_size, shuffle=True)
watch_loader = DataLoader(watch_dataset, batch_size=batch_size, shuffle=True)
loader_by_class.append(train_loader)

input = torch.from_numpy(np.array(X_validation)).to(torch.float32)
label = torch.from_numpy(np.array(validation_labels)).to(torch.long)
torch_dataset = Data.TensorDataset(input, label)
valid_loader = DataLoader(torch_dataset, batch_size=batch_size, shuffle=True)

# input = torch.from_numpy(np.array(X_train)).to(torch.float32)
# label = torch.from_numpy(np.array(true_labels)).to(torch.long)
# input = transform(input)
# torch_dataset = Data.TensorDataset(input,label)
# train_loader = DataLoader(torch_dataset, batch_size=batch_size, shuffle=True)
# loader_by_class.append(train_loader)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified
2000
[array([0., 1., 0., 0., 0., 0., 0., 0., 0., 1.]), array([0., 1., 0., 0., 1., 0., 0., 0., 0., 0.]), array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 1., 0., 0., 1., 0., 0.]), array([0., 0., 0., 0., 0., 0., 1., 1., 0., 0.]), array([1., 0., 1., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 1., 0., 0., 0., 0., 1., 0.]), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 1.]), array([0., 0., 1., 1., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 1., 0., 0., 1., 0., 0.]), array([1., 0., 0., 0., 0., 0., 0., 0., 0., 1.]), array([1., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), array([0., 1., 0., 0., 1., 0., 0., 0., 0., 0.]), array([0., 1., 0., 0., 1., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 1., 1., 0.]), array([0., 0., 0., 1., 0., 1., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 1., 0., 0., 1.]), array([1., 0., 0., 1., 0., 0., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0., 1., 0

# Some Trys

In [ ]:
import scipy.stats

input = np.array([0, 0.5, 0.5])
target = np.array([0, 0.5, 0.5])
print(scipy.stats.entropy(target, input))

0.0


In [ ]:
class Cifar_Net(torch.nn.Module):
    def __init__(self, mode="hidden0",
                 hidden_activation=torch.nn.ReLU):
        super(Cifar_Net, self).__init__()
        self.mode = mode
        if mode == "hidden0":
            self.net = torch.nn.Sequential(
                torch.nn.Linear(3072, 3),
                torch.nn.Softmax(dim=1)
            )
        elif mode == "hidden1":
            self.net = torch.nn.Sequential(
                torch.nn.Linear(3072, 512),
                hidden_activation(),
                torch.nn.Linear(512, 3),
                torch.nn.Softmax(dim=1)
            )
        elif mode == "hidden2":
            self.net = torch.nn.Sequential(
                torch.nn.Linear(3072, 512),
                hidden_activation(),
                torch.nn.Linear(512, 512),
                hidden_activation(),
                torch.nn.Linear(512, 3),
                torch.nn.Softmax(dim=1)
            )
        elif mode == "hidden3":
            self.net = torch.nn.Sequential(
                torch.nn.Linear(3072, 512),
                hidden_activation(),
                torch.nn.Linear(512, 512),
                hidden_activation(),
                torch.nn.Linear(512, 512),
                hidden_activation(),
                torch.nn.Linear(512, 3),
                torch.nn.Softmax(dim=1)
            )
        elif mode == "cnn":
            self.conv = torch.nn.Sequential(
                torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=0),
                hidden_activation(),
                torch.nn.MaxPool2d(kernel_size=2, stride=2),
                torch.nn.Dropout(p=0.25))
            self.fc = torch.nn.Sequential(
                torch.nn.Linear(14400, 512),
                hidden_activation(),
                torch.nn.Dropout(p=0.5),
                torch.nn.Linear(512, 6),
                torch.nn.Softmax(dim=1)
            )
        elif mode == "cnntest":
            self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
            self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
            self.conv2 = torch.nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
            self.fc1 = torch.nn.Linear(in_features=16 * 5 * 5, out_features=120)
            self.fc2 = torch.nn.Linear(in_features=120, out_features=84)
            self.fc3 = torch.nn.Linear(in_features=84, out_features=6)
            self.sm = torch.nn.Softmax(dim=1)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, 0.001)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.1)

    def forward(self, x):
        if self.mode == "cnn":
            x = self.conv(x)
            x = x.flatten(start_dim=1)
            x = self.fc(x)
            return x
        elif self.mode == "cnntest":
            x = self.pool1(torch.nn.functional.relu(self.conv1(x)))
            x = self.pool1(torch.nn.functional.relu(self.conv2(x)))
            x = x.view(-1, 16 * 5 * 5)  # reshape tensor
            x = torch.nn.functional.relu(self.fc1(x))
            x = torch.nn.functional.relu(self.fc2(x))
            x = self.sm(self.fc3(x))
            return x
        else:
            x = self.net(x)
            return x


# NN

In [3]:
import torch.nn as nn
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 10)
        #self.fc4 = nn.Linear(30, 4)
        #self.out = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        #x = self.out(x)
        return x

# NN2

In [ ]:
class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(3*32*32, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 512)
        self.fc4 = nn.Linear(512, 10)
        #self.out = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = self.fc4(x)
        #x = self.out(x)
        return x

# Some scratch

In [ ]:
import torch
a = torch.Tensor([[0.1,0.5,0,0,0,0],[0,0,0.3,0.4,0.3,0]])
b = torch.Tensor([[0,1,0,0,0,0],[0,0,0,1,0,0]])
L = nn.MSELoss()
print(L(a,b))
b = torch.norm(a, p=2, dim=0)
b_2 = torch.norm(a, p=2, dim=1)
b_2 = torch.exp(-b_2)
b_2 = torch.norm(b_2, p=1)/2
print('L2 norm',b)
print('L2 Norm',b_2)

tensor(0.0667)
L2 norm tensor([0.1000, 0.5000, 0.3000, 0.4000, 0.3000, 0.0000])
L2 Norm tensor(0.5794)


In [ ]:
class newLoss(nn.Module):
    def __init__(self, lamda=0.6):
        super(newLoss, self).__init__()
        self.lamda = lamda
        return
    def forward(self, input, targets):
        n = input.shape[0]
        # print('input:', input, 'targets:', targets)
        mse = nn.MSELoss()
        L2Norm = torch.norm(input, p=2, dim=1)/n
        L1L2Norm = torch.norm(L2Norm, p=1)
        Exp_L2 = torch.exp(-L1L2Norm)
        return mse(input, targets)+self.lamda*Exp_L2



# Try Bootstrap Loss Function


In [ ]:
from torch.nn import Module
import torch.nn.functional as F

class SoftBootstrappingLoss(Module):
    """
    ``Loss(t, p) = - (beta * t + (1 - beta) * p) * log(p)``

    Args:
        beta (float): bootstrap parameter. Default, 0.95
        reduce (bool): computes mean of the loss. Default, True.
        as_pseudo_label (bool): Stop gradient propagation for the term ``(1 - beta) * p``.
            Can be interpreted as pseudo-label.
    """
    def __init__(self, beta=0.1, reduce=True, as_pseudo_label=True):
        super(SoftBootstrappingLoss, self).__init__()
        self.beta = beta
        self.reduce = reduce
        self.as_pseudo_label = as_pseudo_label

    def forward(self, y_pred, y):
        # cross_entropy = - t * log(p)
        beta_xentropy = self.beta * F.cross_entropy(y_pred, y, reduction='none')

        y_pred_a = y_pred.detach() if self.as_pseudo_label else y_pred
        # second term = - (1 - beta) * p * log(p)
        bootstrap = - (1.0 - self.beta) * torch.sum(F.softmax(y_pred_a, dim=1) * F.log_softmax(y_pred, dim=1), dim=1)

        if self.reduce:
            return torch.mean(beta_xentropy + bootstrap)
        return beta_xentropy + bootstrap


class HardBootstrappingLoss(Module):
    """
    ``Loss(t, p) = - (beta * t + (1 - beta) * z) * log(p)``
    where ``z = argmax(p)``

    Args:
        beta (float): bootstrap parameter. Default, 0.95
        reduce (bool): computes mean of the loss. Default, True.

    """
    def __init__(self, beta=0.8, reduce=True):
        super(HardBootstrappingLoss, self).__init__()
        self.beta = beta
        self.reduce = reduce

    def forward(self, y_pred, y):
        # cross_entropy = - t * log(p)
        beta_xentropy = self.beta * F.cross_entropy(y_pred, y, reduction='none')

        # z = argmax(p)
        z = F.softmax(y_pred.detach(), dim=1).argmax(dim=1)
        z = z.view(-1, 1)
        bootstrap = F.log_softmax(y_pred, dim=1).gather(1, z).view(-1)
        # second term = (1 - beta) * z * log(p)
        bootstrap = - (1.0 - self.beta) * bootstrap

        if self.reduce:
            return torch.mean(beta_xentropy + bootstrap)
        return beta_xentropy + bootstrap


In [ ]:
from torch.nn import Module
import torch.nn.functional as F

a = torch.Tensor([[0.1,0.5,0,0,0,0],[0,0,0.3,0.4,0.3,0]])
label = torch.Tensor([[[0,1,0,0,0,0],[0,0,0,1,0,0]],[[1,0,0,0,0,0],[0,0,0,0,1,0]]])
print(label.shape)
b = label[0]
c = label[1]
l1 = F.cross_entropy(a, b, reduction='none')
l2 = F.cross_entropy(a, c, reduction='none')

print(l1)
print(l2)
l = torch.stack((l1,l2),0)
print(l)
l_sum = torch.sum(l, dim=0)
print(l_sum)
l_min = torch.min(l, dim=0).values
print(l_min)

torch.Size([2, 2, 6])
tensor([1.4101, 1.5729])
tensor([1.8101, 1.6729])
tensor([[1.4101, 1.5729],
        [1.8101, 1.6729]])
tensor([3.2202, 3.2458])
tensor([1.4101, 1.5729])


In [4]:
import torch
from torch.nn import Module
import torch.nn.functional as F


class LogEntropyLoss(Module):
    # a loss for subset targets data
    def __init__(self):
        super(LogEntropyLoss, self).__init__()

    def forward(self, logits, target):
        # logits format:[[a,b,c,d],[e,f,g,h]]
        # target format:[[1,1,1,0],[0,1,1,0]]
        logits = logits.contiguous()  # [NHW, C]
        logits = F.softmax(logits, 1)
        mask = target.bool()
        logits = logits.masked_fill(~mask, 0)
        loss = torch.sum(logits, dim=1)
        loss = torch.log(loss)
        loss = -1 * loss
        return(loss.mean())

In [ ]:
from torch.nn import Module
import torch.nn.functional as F

class MinEntropyLoss(Module):
    def __init__(self):
        super(MinEntropyLoss, self).__init__()
    def forward(self, y_pred, y):
        # #print('Loss Test:')
        # print('y_pred',y_pred)
        # print('y_label', y)
        y_classes = y.shape[0]
        loss = torch.Tensor([0.])
        loss_0 = F.cross_entropy(y_pred, y[0], reduction='mean')
        loss_1 = F.cross_entropy(y_pred, y[1], reduction='mean')
        #return loss_0 + loss_1
        loss_stack = torch.stack((loss_0, loss_1), 0)
        for i in range(2, y_classes):
            loss = F.cross_entropy(y_pred, y[i])
            loss_stack = torch.stack((loss_stack, loss), 0)
        #print('loss_stack', loss_stack.shape, loss_stack)
        max_loss = torch.max(loss_stack, dim=0).values
        #sum_loss = torch.sum(loss_stack, dim=0)
        # print('min_loss', min_loss)
        return torch.mean(max_loss)




# Mixup method

In [ ]:
from torch.nn import Module
import torch.nn.functional as F


def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
        lam = max(1-lam, lam)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b) 


# Train NN

In [9]:
from torchvision import models
epochs = 48
learning_rate = 0.001

# modes = ["hidden2","hidden3"]
mode = "cnntest"
import matplotlib.pyplot as plt

def train(train_loader, X_validation):
    torch.cuda.empty_cache()
    # model = models.resnet18(pretrained=True)
    # fc_in = model.fc.in_features  # 获取全连接层的输入特征维度
    # model.fc = nn.Linear(fc_in, 10)
    model = Net()
    if torch.cuda.is_available():
        model = model.cuda()
    # 定义自己的优化器                      
    # 定义损失函数和优化器
    # criterion = HardBootstrappingLoss(beta=0.9)
    # criterion = SoftBootstrappingLoss()
    criterion = LogEntropyLoss()
    watch_criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9,
                       #weight_decay=1e-4)
    train_loss = []
    validation_accuracy = []
    true_loss = []
    for epoch in range(epochs):
        sum_loss = 0.0
        for data in train_loader:
            img, label = data
            img = transform(img)
            if torch.cuda.is_available():
                img = Variable(img).cuda()
                label = Variable(label).cuda()
            else:
                img = Variable(img)
                label = Variable(label)
            out = model(img)
            loss = criterion(out, label)
            print_loss = loss.data.item()
            sum_loss += print_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss.append(sum_loss)
        print('Epoch', epoch, 'Loss:', sum_loss)
        # 监视真实损失
        sum_loss = 0.0
        with torch.no_grad():
            for data in watch_loader:
                val_inputs, label = data
                val_inputs = transform(val_inputs)
                if torch.cuda.is_available():
                    val_inputs = Variable(val_inputs).cuda()
                    label = Variable(label).cuda()
                val_outputs = model(val_inputs)
                loss = watch_criterion(val_outputs, label)
                print_loss = loss.data.item()
                sum_loss += print_loss
            print('Epoch:', epoch, 'True Loss:', sum_loss)
            true_loss.append(sum_loss)       
        # 监视验证集
        total_correct = 0.0
        total_num = 0.0
        with torch.no_grad():
            for data in valid_loader:
                val_inputs, label = data
                val_inputs = transform(val_inputs)
                if torch.cuda.is_available():
                    val_inputs = Variable(val_inputs).cuda()
                    label = Variable(label).cuda()
                val_outputs = model(val_inputs)
                #print(val_outputs)
                #print(validation_labels)
                pred = val_outputs.argmax(dim=1)
                total_correct += torch.eq(pred,label).float().sum().item() #分别为是否相等，scalar tensor转换为float，求和，拿出值
                total_num += label.size(0)
            acc = total_correct/total_num
            print('Epoch:', epoch, 'Val Acc:', acc)
            validation_accuracy.append(acc)       
    epoch_num = epochs
    x1 = range(0, epoch_num)
    x2 = range(0, epoch_num)
    x3 = range(0, epoch_num)
    y1 = validation_accuracy
    y2 = train_loss
    y3 = true_loss
    plt.subplot(2, 1, 1)
    plt.plot(x1, y1, 'o-')
    plt.title('Validation indicators vs. epoches')
    plt.ylabel('Validation accuracy')
    plt.subplot(2, 1, 2)
    plt.plot(x2, y2, '.-')
    plt.xlabel('epochs')
    plt.ylabel('Train loss')
    plt.subplot(2, 1, 3)
    plt.plot(x3, y3, '.-')
    plt.xlabel('epochs')
    plt.ylabel('True loss')
    # plt.savefig('Validation indicators '+mode+".png")
    plt.show()
    #return model



In [ ]:
from torchvision import models
epochs = 128
learning_rate = 0.001

# modes = ["hidden2","hidden3"]
mode = "cnntest"
import matplotlib.pyplot as plt

def train_cnn(train_loader, X_validation):
    model = Net2()
    if torch.cuda.is_available():
        model = model.cuda()
    # 定义自己的优化器                      
    # 定义损失函数和优化器
    # criterion = HardBootstrappingLoss(beta=0.9)
    # criterion = SoftBootstrappingLoss()
    criterion = LogEntropyLoss()
    #criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9,
                       #weight_decay=1e-4)
    train_loss = []
    validation_accuracy = []
    for epoch in range(epochs):
        sum_loss = 0.0
        for data in train_loader:
            # data_size = data[0].shape[0]
            # use_cuda = torch.cuda.is_available()
            img, label = data
            #img = transform(img)
            #label = label.reshape(2, -1, 4)
            # if use_cuda:
            #     img, label = img.cuda(), label.cuda()
            # imgs, labels_a, labels_b, lam = mixup_data(img, label,
            #                                            0.75, use_cuda)
            # imgs, labels_a, labels_b = map(Variable, (imgs, labels_a, labels_b))
            # outputs = net(inputs)
            if torch.cuda.is_available():
                img = Variable(img).cuda()
                label = Variable(label).cuda()
            else:
                img = Variable(img)
                label = Variable(label)
            out = model(img)
            # loss = mixup_criterion(criterion, out, labels_a, labels_b, lam)
            loss = criterion(out, label)
            print_loss = loss.data.item()
            #print('loss',print_loss)
            sum_loss += print_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss.append(sum_loss)
        # 监视验证集
        total_correct = 0.0
        total_num = 0.0
        with torch.no_grad():
            for data in valid_loader:
                val_inputs, label = data
                #val_inputs = transform(val_inputs)
                if torch.cuda.is_available():
                    val_inputs = Variable(val_inputs).cuda()
                    label = Variable(label).cuda()
                val_outputs = model(val_inputs)
                #print(val_outputs)
                #print(validation_labels)
                pred = val_outputs.argmax(dim=1)
                total_correct += torch.eq(pred,label).float().sum().item() #分别为是否相等，scalar tensor转换为float，求和，拿出值
                total_num += label.size(0)
            acc = total_correct/total_num
            print('Epoch:', epoch, 'Val Acc:', acc)
            validation_accuracy.append(acc)       
        print('Epoch', epoch, 'Loss:', sum_loss)
    epoch_num = epochs
    x1 = range(0, epoch_num)
    x2 = range(0, epoch_num)
    y1 = validation_accuracy
    y2 = train_loss
    plt.subplot(2, 1, 1)
    plt.plot(x1, y1, 'o-')
    plt.title('Validation indicators vs. epoches')
    plt.ylabel('Validation accuracy')
    plt.subplot(2, 1, 2)
    plt.plot(x2, y2, '.-')
    plt.xlabel('epochs')
    plt.ylabel('Train loss')
    # plt.savefig('Validation indicators '+mode+".png")
    plt.show()
    #return model



In [ ]:
def train_model(train_loader, X_validation, model):
    # model = Net()
    # model = models.resnet18(pretrained=True)
    # fc_in = model.fc.in_features  
    # model.fc = nn.Linear(fc_in, 6)                        
    if torch.cuda.is_available():
        model = model.cuda()
    # 定义损失函数和优化器
    # criterion = HardBootstrappingLoss(beta=0.9)
    # criterion = SoftBootstrappingLoss(beta=0.99)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    # #optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9,
    #                   weight_decay=1e-4)
    train_loss = []
    validation_accuracy = []
    for epoch in range(epochs):
        sum_loss = 0.0
        for data in train_loader:
            # data_size = data[0].shape[0]
            # use_cuda = torch.cuda.is_available()
            img, label = data
            # if use_cuda:
            #     img, label = img.cuda(), label.cuda()
            # imgs, labels_a, labels_b, lam = mixup_data(img, label,
            #                                            0.75, use_cuda)
            # imgs, labels_a, labels_b = map(Variable, (imgs, labels_a, labels_b))
            # outputs = net(inputs)
            if torch.cuda.is_available():
                img = Variable(img).cuda()
                label = Variable(label).cuda()
            else:
                img = Variable(img)
                label = Variable(label)
            out = model(img)
            # loss = mixup_criterion(criterion, out, labels_a, labels_b, lam)
            loss = criterion(out, label)
            print_loss = loss.data.item()
            sum_loss += print_loss
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss.append(sum_loss)
        # 监视验证集
        with torch.no_grad():
            val_input = torch.from_numpy(np.array(X_validation)).to(torch.float32)
            val_inputs = val_input.reshape(-1, 3, 32, 32)
            if torch.cuda.is_available():
                val_inputs = Variable(val_inputs).cuda()
            val_outputs = model(val_inputs)
            print('Epoch:', epoch, 'output')
            #print(val_outputs)
            #print(validation_labels)
            val_preds = np.argmax(val_outputs.cpu().detach().numpy(), axis=1)
            acc = accuracy_score(val_preds, validation_labels)
            print('Epoch:', epoch, 'Val Acc:', acc)
            validation_accuracy.append(acc)       
        print('Epoch', epoch, 'Loss:', sum_loss)
    epoch_num = epochs
    x1 = range(0, epoch_num)
    x2 = range(0, epoch_num)
    y1 = validation_accuracy
    y2 = train_loss
    plt.subplot(2, 1, 1)
    plt.plot(x1, y1, 'o-')
    plt.title('Validation indicators vs. epoches')
    plt.ylabel('Validation accuracy')
    plt.subplot(2, 1, 2)
    plt.plot(x2, y2, '.-')
    plt.xlabel('epochs')
    plt.ylabel('Train loss')
    # plt.savefig('Validation indicators '+mode+".png")
    plt.show()
    #return model

In [ ]:
#torch.cuda.empty_cache()
train(loader_by_class[-1], X_validation)

Epoch 0 Loss: 551.8815870285034
Epoch: 0 True Loss: 703.4015868902206
Epoch: 0 Val Acc: 0.3195
Epoch 1 Loss: 507.5841910839081
Epoch: 1 True Loss: 661.8292944431305
Epoch: 1 Val Acc: 0.346
Epoch 2 Loss: 491.96756076812744
Epoch: 2 True Loss: 648.9511551856995
Epoch: 2 Val Acc: 0.3535
Epoch 3 Loss: 482.2780486345291
Epoch: 3 True Loss: 633.1979689598083
Epoch: 3 Val Acc: 0.36
Epoch 4 Loss: 477.5848762989044
Epoch: 4 True Loss: 632.3542598485947
Epoch: 4 Val Acc: 0.37
Epoch 5 Loss: 473.45736253261566
Epoch: 5 True Loss: 632.7855542898178
Epoch: 5 Val Acc: 0.365
Epoch 6 Loss: 468.18403136730194
Epoch: 6 True Loss: 609.140095949173
Epoch: 6 Val Acc: 0.3975
Epoch 7 Loss: 464.4824985265732
Epoch: 7 True Loss: 621.0218189954758
Epoch: 7 Val Acc: 0.38
Epoch 8 Loss: 461.39355170726776
Epoch: 8 True Loss: 611.3112077713013
Epoch: 8 Val Acc: 0.398
Epoch 9 Loss: 459.2201980352402
Epoch: 9 True Loss: 598.4882682561874
Epoch: 9 Val Acc: 0.3985
Epoch 10 Loss: 456.14846992492676
Epoch: 10 True Loss: 5

In [ ]:
X_validation = []
validation_labels = []
for index,y in enumerate(validation_y):
    if y < 6:
        X_validation.append(validation_X[index])
        validation_labels.append(y)
with torch.no_grad():
    val_input = torch.from_numpy(np.array(X_validation)).to(torch.float32)
    val_inputs = val_input.reshape(-1, 3, 32, 32)
    if torch.cuda.is_available():
        val_inputs = Variable(val_inputs).cuda()
    val_outputs = model(val_inputs)
    print(val_outputs, validation_labels)
    # print('Epoch:', epoch, 'output')
    #print(val_outputs)
    #print(validation_labels)
    val_preds = np.argmax(val_outputs.cpu().detach().numpy(), axis=1)
    acc = accuracy_score(val_preds, validation_labels)
    print('Val Acc:', acc)
     


tensor([[-0.2589, -0.1860, -0.1345, -0.1278,  0.2907, -0.1038],
        [-0.2014, -0.1336, -0.1388,  0.2011, -0.2499,  0.0287],
        [-0.1282, -0.2026, -0.0512, -0.1403,  0.1437, -0.1621],
        ...,
        [-0.2200, -0.2588, -0.0906, -0.0760,  0.1529, -0.0086],
        [-0.3775, -0.4570,  0.1052, -0.1178,  0.2476, -0.1434],
        [-0.2920, -0.3247, -0.1502, -0.2001, -0.2513,  0.5103]],
       device='cuda:0') [2, 1, 5, 3, 0, 1, 5, 1, 3, 5, 2, 3, 4, 0, 2, 5, 2, 4, 5, 5, 5, 0, 4, 1, 2, 3, 2, 2, 3, 5, 5, 0, 1, 2, 2, 2, 1, 4, 2, 0, 0, 4, 5, 3, 1, 2, 3, 4, 3, 1, 1, 5, 1, 5, 3, 3, 4, 3, 5, 1, 1, 2, 1, 2, 5, 0, 2, 2, 3, 1, 4, 2, 0, 3, 2, 0, 3, 1, 0, 0, 1, 5, 3, 4, 5, 5, 1, 5, 4, 3, 5, 3, 5, 0, 4, 3, 0, 0, 0, 4, 1, 4, 4, 2, 0, 0, 1, 2, 2, 3, 3, 0, 4, 1, 3, 4, 5, 1, 3, 2, 2, 4, 2, 3, 0, 2, 3, 4, 0, 3, 5, 3, 0, 1, 2, 3, 5, 1, 0, 3, 4, 1, 3, 1, 0, 1, 2, 5, 2, 3, 4, 5, 5, 3, 3, 2, 5, 4, 0, 5, 3, 3, 3, 4, 0, 0, 2, 4, 3, 1, 3, 1, 3, 2, 1, 5, 3, 0, 3, 1, 2, 2, 1, 5, 0, 0, 4, 5, 4, 1, 5, 5, 0

In [ ]:
model = train_model(loader_by_class[-1], X_validation, model)

Epoch: 0 output
Epoch: 0 Val Acc: 0.3757628596338274
Epoch 0 Loss: 762.9999468326569
Epoch: 1 output
Epoch: 1 Val Acc: 0.36965998256320837
Epoch 1 Loss: 761.0371149778366
Epoch: 2 output
Epoch: 2 Val Acc: 0.37401918047079336
Epoch 2 Loss: 759.6408988237381
Epoch: 3 output
Epoch: 3 Val Acc: 0.3618134263295554
Epoch 3 Loss: 758.5061184167862
Epoch: 4 output
Epoch: 4 Val Acc: 0.35658238884045335
Epoch 4 Loss: 757.2539565563202
Epoch: 5 output
Epoch: 5 Val Acc: 0.36268526591107236
Epoch 5 Loss: 756.0671961307526


KeyboardInterrupt: ignored

In [ ]:
total_test_outputs = []
total_classes = 6
import scipy.stats

def benchmark_dist():
    benchmark = np.zeros(shape=(total_classes,total_classes))
    for i in range(0, total_classes):
        for j in range(0, total_classes):
            if i == j:
                benchmark[i][j] = 1/3
            else:
                benchmark[i][j] = 2/15
    return benchmark


def test_total_outputs_to_preds(test_outputs, benchmark):
    data_size = test_outputs[0].shape[0]
    test_preds = np.zeros(shape=(total_classes, data_size))
    print(test_preds)
    for i in range(0, data_size):
        for j in range(0, total_classes):
            test_preds[j][i] = scipy.stats.entropy(test_outputs[j][i], benchmark[j])
    print(test_preds)
    test_preds = np.argmin(test_preds, axis=0)
    print(test_preds)
    return test_preds     

for train_loader in loader_by_class:
    model = train(train_loader, X_validation)
    with torch.no_grad():
        test_input = torch.from_numpy(np.array(X_test)).to(torch.float32)
        test_inputs = test_input.reshape(-1, 3, 32, 32)
        if torch.cuda.is_available():
            test_inputs = Variable(test_inputs).cuda()
        test_outputs = model(test_inputs).cpu().detach().numpy()
        total_test_outputs.append(test_outputs)

benchmark = benchmark_dist()
test_preds = test_total_outputs_to_preds(total_test_outputs, benchmark)
        #test_preds = np.argmax(test_outputs.cpu().detach().numpy(), axis=1)
print(mode, (accuracy_score(test_preds, test_labels)))


NameError: ignored